# MentaLiST 0.2

MentaLiST 0.2 has a new calling algorithm and also detects and reconstructs putative novel alleles, also calling non-present loci, allowing the use for wgMLST schemes.



## Downloading the new version

MentaLiST 0.2 is on a different branch on the github repository. You can clone the repository as usual, with the -b flag to get the new branch: 

In [1]:
git clone https://github.com/WGS-TB/MentaLiST -b mentalist_v0.2

Cloning into 'MentaLiST'...
remote: Counting objects: 800, done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 800 (delta 15), reused 18 (delta 9), pack-reused 775
Receiving objects: 100% (800/800), 37.89 MiB | 14.49 MiB/s, done.
Resolving deltas: 100% (428/428), done.


You can create an alias to make it easier to access this version, specially if you also have the original MentaLiST version installed with bioconda.

In [2]:
alias mentalist="julia --depwarn=no $PWD/MentaLiST/src/mentalist"

MLST calling has some new options, in relation the the previous version: 

In [3]:
mentalist call -h

usage: mentalist call -o O -s S --db DB [-t MUTATION_THRESHOLD]
                      [--kt KT] [--output_votes] [--output_special]
                      [-h] files...

positional arguments:
  files                 FastQ input files

optional arguments:
  -o O                  Output file with MLST call
  -s S                  Sample name
  --db DB               Kmer database
  -t, --mutation_threshold MUTATION_THRESHOLD
                        Maximum edit distance (number of mutations)
                        when looking for novel alleles. (type: Int64,
                        default: 6)
  --kt KT               Minimum # of times a kmer is seen, to be
                        considered 'solid', meaning actually present
                        in the sample. (type: Int64, default: 10)
  --output_votes        Also outputs the results for the original
                        voting algorithm, without novel.
  --output_special      Also outputs a FASTA file with the alleles
           

The -t is the maximum distance, in number of mutations (nucleotide substitutions, insertions and/or deletions) that MentaLiST will apply to an existing allele, while searching for a novel allele present in a sample. Larger values might take considerably longer.

The --kt option determines the minimum number of times that a kmer has to be observed in the FASTQ sample to be considered 'solid'. This will be used on the calling phase, to determine if a particular allele is present (all its k-mers have to be present in the sample and above the --kt threshold), and also in the search for novel alleles. You might increase this value if you have larger depth in your sample.

The --output_votes enables the output of the MLST calling files for the old algorithm, based only on the maximum vote count, without considering allele coverage.

When --output_special is given, a FASTA file with all the 'special cases' will be created. This includes loci where no allele has 100% coverage, or there is more than one allele with 100% coverage, and also novel alleles. 

## Running MentaLiST 0.2

Because of the new calling algorithm, new information has to be stored on the MentaLiST database, so databases created with the previous version are not compatible. The command for creating a database is exactly the same as before.

In [4]:
mentalist build_db -h

usage: mentalist build_db --db DB -k K -f FASTA_FILES [FASTA_FILES...]
                        [-p PROFILE] [-c] [-h]

optional arguments:
  --db DB               Output file (kmer database)
  -k K                  Kmer size (type: Int8)
  -f, --fasta_files FASTA_FILES [FASTA_FILES...]
                        Fasta files with the MLST scheme
  -p, --profile PROFILE
                        Profile file for known genotypes.
  -c, --disable_compression
                        Disables the default compression of the
                        database, that stores only the most
                        informative kmers. Not recommended unless for
                        debugging.
  -h, --help            show this help message and exit



In this example, the folder 'MTB_scheme' has around 100 FASTA files, a subset from a large M. tuberculosis cgMLST scheme with around 3000 loci. To build a MentaLiST database for this scheme, similarly as the previous MentaLiST version, run:   

In [5]:
mentalist build_db --db mtb_cgMLST.db -k 31 -f MTB_scheme/*.fa

2017-11-23T18:10:48.178 - info: Opening FASTA files ... 
2017-11-23T18:11:02.908 - info: Combining results for each locus ...
2017-11-23T18:11:03.37 - info: Saving DB ...
2017-11-23T18:11:05.172 - info: Done!


In [6]:
ls -lh mtb_cgMLST.db

-rw-r--r-- 1 pfeijao users 1.5M Nov 23 18:11 mtb_cgMLST.db


Now, on this example we have a sample.fastq.gz file with a WGS sample from M. tuberculosis. To run the MLST caller: 

In [7]:
mentalist call -o sample.call -s sample --db mtb_cgMLST.db --kt 10 --output_votes --output_special sample.fastq.gz

2017-11-23T18:11:13.916 - info: Opening kmer database ... 
2017-11-23T18:11:17.394 - info: Opening fastq file(s) and counting kmers ... 
2017-11-23T18:12:17.454 - info: Voting for alleles ... 
2017-11-23T18:12:18.174 - info: Calling alleles and novel alleles ...
2017-11-23T18:12:22.886 - info: Writing output ...
2017-11-23T18:12:23.725 - info: Done.


### Description of output files

Here, a brief description of each output file. All output files have the same prefix, given by the -o option when running MentaLiST call. 'sample.call' in this example.


In [8]:
ls -l sample.call*

-rw-r--r-- 1 pfeijao users   720 Nov 23 18:12 sample.call
-rw-r--r-- 1 pfeijao users   721 Nov 23 18:12 sample.call.byvote
-rw-r--r-- 1 pfeijao users  2714 Nov 23 18:12 sample.call.coverage.txt
-rw-r--r-- 1 pfeijao users  3209 Nov 23 18:12 sample.call.novel.fa
-rw-r--r-- 1 pfeijao users   188 Nov 23 18:12 sample.call.novel.txt
-rw-r--r-- 1 pfeijao users 15143 Nov 23 18:12 sample.call.special_cases.fa
-rw-r--r-- 1 pfeijao users   128 Nov 23 18:12 sample.call.ties.txt
-rw-r--r-- 1 pfeijao users 14079 Nov 23 18:12 sample.call.votes.txt


In [9]:
# The 'main' fil is sample.call, with the allele calls:
cat sample.call

Sample	Rv0014c	Rv0015c	Rv0016c	Rv0017c	Rv0018c	Rv0019c	Rv0021c	Rv0022c	Rv0023	Rv0024	Rv0025	Rv0033	Rv0034	Rv0035	Rv0036c	Rv0037c	Rv0038	Rv0039c	Rv0040c	Rv0041	Rv0042c	Rv0043c	Rv0044c	Rv0045c	Rv0046c	Rv0047c	Rv0048c	Rv0049	Rv0050	Rv0051	Rv0052	Rv0053	Rv0054	Rv0055	Rv0056	Rv0058	Rv0059	Rv0060	Rv0061c	Rv0062	Rv0063a	Rv0063	Rv0065	Rv0066c	Rv0067c	Rv0068	Rv0069c	Rv0070c	Rv0075	Rv0076c	Rv0077c	Rv0078A	Rv0078B	Rv0078	Rv0079	Rv0080	Rv0081	Rv0082	Rv0084	Rv0085	Rv0086	Rv0087	Rv0089	Rv0090	Rv0091	Rv0092	Rv0097	Rv0098	Rv0099	Rv0767c	Rv0768	Rv1269c	ST	clonal_complex
sample	5	2	1	1	2	1	1	1	1	N	1	1	2	2	6	1	1	2	2	6	1	1	2	N	2	1	4	1	4	1	1	1	1	1	1	3	1	1+	2	1	1	N	1	1	1	2	1	1	1	1	1	3	1	1	1	1	1	1	1	1	3	1	1	3	1	4	1	1	1+	0	0	1/N?	0	0


The file sample.call.coverage.txt has a description of each the call for each locus. There are different types of call possible:


In [10]:
# 'Regular' called alleles, the most voted allele that has 100% coverage; this is the most common case.
grep Called sample.call.coverage.txt | head -n5

Rv0014c	1	27	Called allele 5.
Rv0015c	1	27	Called allele 2.
Rv0016c	1	27	Called allele 1.
Rv0017c	1	22	Called allele 1.
Rv0018c	1	28	Called allele 2.


In [11]:
# Non present loci; might have some k-mers seen, but below threshold, so it is declared missing.
grep "Not present" sample.call.coverage.txt

Rv0767c	0.5376	0	Not present; allele 18 is the best voted but below threshold with 283/612 missing kmers.
Rv0768	0.1684	0	Not present; allele 20 is the best voted but below threshold with 1195/1437 missing kmers.


In [12]:
# Novel alleles; no existing allele has 100% coverage, so MentaLiST looks for variations that have 100% coverage,
# using existing alleles as "template" for creating a novel allele.
grep Novel sample.call.coverage.txt

Rv0024	1	20	Novel, 1 mutation from allele 1: Del of len 1 at pos 6
Rv0045c	1	27	Novel, 1 mutation from allele 185: Ins of base G at pos 373
Rv0063	1	33	Novel, 1 mutation from allele 1: Ins of base C at pos 1417


In [13]:
# Multiple possible alleles: when more than one allele has 100% coverage. 
# Here, the depth of coverage and number of votes of each allele is shown; the best one is chosen on the call file, 
# with a flag "+" after the allele number.
grep Multiple sample.call.coverage.txt

Rv0060	1	*	Multiple possible alleles:1, 2 with depth 31, 16 and votes 0, -443. Most voted (1) is chosen on call file.
Rv0099	1	*	Multiple possible alleles:1, 4 with depth 31, 12 and votes 0, -690. Most voted (1) is chosen on call file.


In [14]:
# Partially covered alleles, where no novel allele was found; Most likely either an undetected novel allele or 
# an existing allele that was not fully covered in the WGS sample, for some reason. 
grep Partially sample.call.coverage.txt

Rv1269c	0.9449	0	Partially covered alelle, novel or not present; Most covered allele 1 has 19/345 missing kmers, and no novel was found.


#### Novel alleles output

Two files: one FASTA with the sequences, and one with a description.

In [15]:
# novel alleles found:
cat sample.call.novel.fa

>Rv0024
GTGAATACAGCGAGGTCGAGCTGTTGAGTCGCGCTCATCAACTGTTCGCCGGAGACAGTCGGCGACCGGGGTTGGATGCGGGCACCACACCCTACGGGGATCTGCTGTCTCGGGCTGCCGACCTGAATGTGGGTGCGGGCCAGCGCCGGTATCAACTCGCCGTGGACCACAGCCGGGCGGCCTTGCTGTCTGCTGCGCGAACCGATGCCGCGGCCGGGGCCGTCATCACCGGCGCTCAACGGGATCGGGCATGGGCCCGGCGGTCGACCGGAACCGTTCTCGACGAGGCTCGCTCGGATACCACCGTTACTGCGGTTATGCCGATAGCCCAGCGCGAAGCCATACGCCGTCGTGTGGCGCGGCTGCGCGCGCAACGAGCCCATGTGCTGACGGCGCGACGACGGGCACGACGGCACCTGGCGGCGCTGCGTGCGCTGCGGTACCGGGTGGCGCACGGCCCGGGGGTCGCGCTGGCCAAACTTCGGCTGCCGTCGCCGAGCGGTCGCGCCGGCATCGCGGTCCACGCCGCGCTGTCGCGACTTGGCCGTCCCTATGTCTGGGGCGCAACGGGGCCCAACCAGTTCGACTGTTCCGGTTTGGTCCAGTGGGCCTACGCCCAGGCGGGTGTTCACCTGGATCGCACCACCTATCAACAGATCAACGAGGGGATCCCGGTGCCGCGCTCACAGGTCCGGCCGGGCGATCTGGTCTTCCCGCACCCCGGGCACGTGCAGCTGGCGATCGGCAACAATCTGGTCGTCGAGGCGCCCCATGCGGGCGCGTCGGTTCGGGTCAGCTCGCTGGGCAACAACGTGCAGATTCGGCGACCGCTGAGTGGCAGATAA
>Rv0045c
TCAGCGTGTGTCGAGCACCCCGCGCACGATCTCGATCAGGGCGCGCGGTTGGTCACTTTGCACCGAGTGGCCTGACTTCTCGACGATGTGAACGCCACGGAAATGCGTTGCACGCCTGTGGAGTTCGGCGGTGTCCT

In [16]:
# Description of novel alleles; number of mutations, and description of the mutation;
cat sample.call.novel.txt

Loci	MinKmerDepth	Nmut	Desc
Rv0024	20	1	From allele 1, Del of len 1 at pos 6.
Rv0045c	27	1	From allele 185, Ins of base G at pos 373.
Rv0063	33	1	From allele 1, Ins of base C at pos 1417.


In [17]:
# Alternative call, without novel/missing genes algorithm, purely by votes; 
# only included if --output-votes is specified.
cat sample.call.byvote 

Sample	Rv0014c	Rv0015c	Rv0016c	Rv0017c	Rv0018c	Rv0019c	Rv0021c	Rv0022c	Rv0023	Rv0024	Rv0025	Rv0033	Rv0034	Rv0035	Rv0036c	Rv0037c	Rv0038	Rv0039c	Rv0040c	Rv0041	Rv0042c	Rv0043c	Rv0044c	Rv0045c	Rv0046c	Rv0047c	Rv0048c	Rv0049	Rv0050	Rv0051	Rv0052	Rv0053	Rv0054	Rv0055	Rv0056	Rv0058	Rv0059	Rv0060	Rv0061c	Rv0062	Rv0063a	Rv0063	Rv0065	Rv0066c	Rv0067c	Rv0068	Rv0069c	Rv0070c	Rv0075	Rv0076c	Rv0077c	Rv0078A	Rv0078B	Rv0078	Rv0079	Rv0080	Rv0081	Rv0082	Rv0084	Rv0085	Rv0086	Rv0087	Rv0089	Rv0090	Rv0091	Rv0092	Rv0097	Rv0098	Rv0099	Rv0767c	Rv0768	Rv1269c	ST	clonal_complex
sample	5	2	1	1	2	1	1	1	1	165	1	1	2	2	6	1	1	2	2	6	1	1	2	7	2	1	4	1	4	1	1	1	1	1	1	3	1	1	2	1	1	248	1	1	1	2	1	1	1	1	1	3	1	1	1	1	1	1	1	1	3	1	1	3	1	4	1	1	1	18	74	1	0	0


## Updating a database with the detected novel alleles

You can update your MLST scheme with the novel alleled detected by MentaLiST, specially after running it on many different samples. In the scripts folder, there are python scripts to help select alleles and build an updated scheme. To do that, you will perform three steps:
1. Select a subset of all novel alleles that satisfy some restrictions.
1. Create a new MLST from the existing one, by adding the novel alleles.
1. Run MentaLiST to create a k-mer database for this MLST scheme.

Each step is described below.

### Selecting the novel alleles

In [18]:
# optional: select the python environment to run the scripts;
PYTHON=~/.conda/envs/pathogist/bin/python

The 'parse_novel_alleles.py' script collects all novel alleles, creates a report and also outputs a FASTA file with selected alleles, to include in an updated MLST scheme. 

In [19]:
$PYTHON ../scripts/parse_novel_alleles.py -h

usage: parse_novel_alleles.py [-h] [-f F [F ...]] [-o O] [-t THRESHOLD]
                              [-m MUTATION]
                              [-ll {DEBUG,INFO,WARNING,ERROR,CRITICAL}]

Given a list of FASTA files with novel alleles found with MentaLiST, output a
FASTA with a unique list of novel alleles.

optional arguments:
  -h, --help            show this help message and exit
  -f F [F ...]          Fasta files with novel alleles.
  -o O                  Output Fasta file with alleles above the threshold
                        requirement(s).
  -t THRESHOLD, --threshold THRESHOLD
                        Minimum number of different samples to appear, to
                        include a novel allele in the output fasta.
  -m MUTATION, --mutation MUTATION
                        Also include if novel allel has equal or less than
                        this number of mutations, regardless of times seen.
                        Disabled by default.
  -ll {DEBUG,INFO,WARNING,ERROR

You must give all the novel allele FASTA files found by MentaLiST as parameter -f. Any given novel allele will be included in the output file (parameter -o) if this exact allele is present in at least (-t) samples. Also, if the parameter -m is given, any novel allele that has -m or less mutations is also included, if you want to include any allele that is very close to existing alleles. 

In the following example, I have a folder 'results' with MentaLiST results for >1000 samples. I want to select only alleles that have been detected in at least 5 samples.

In [20]:
$PYTHON ../scripts/parse_novel_alleles.py -f results/*novel.fa -o new_alleles.fa -t 5 > new_alleles_report.txt

06:12:26 PM (178 ms) -> INFO:Reading the new alleles  ...
06:12:30 PM (3953 ms) -> INFO:Writing output ...
06:12:30 PM (4029 ms) -> INFO:Done.


The report has one line for each locus, with the total number of alleles, how many times each allele is present on all samples and the number of mutations in relation to an existing allele.

In [21]:
head new_alleles_report.txt

Locus	Alleles found	Samples x (mutations)
Rv0014c	24	5x (1), 5x (1), 2x (1), 2x (1), 2x (1), 2x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (2), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1)
Rv0015c	16	2x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1)
Rv0016c	17	6x (1), 2x (1), 2x (1), 2x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (2), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1)
Rv0017c	30	8x (1), 2x (1), 2x (1), 2x (1), 2x (1), 2x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1)
Rv0018c	19	7x (1), 2x (1), 2x (1), 1x (1), 1x (1), 1x (1), 1x (2), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1), 1x (1)
Rv0019c	6	8x (1), 3x (1), 1x (1), 1x (1), 1x (1), 1x (1)
Rv0

For instance, there are 14 novel alleles for locus Rv0021c, the two most common being present in 88 and 5 samples, respectively. Both are 1 mutation away from an existing allele.

The FASTA output will have all alleles that have been seen at least on 5 samples. So we know that for locus Rv0021c we will have two alleles:

In [22]:
grep Rv0021 new_alleles.fa -A1

>Rv0021c
TTACACGGCGCGCATCGCGGTCAGCGTCCTCGCCGCTTGTTGCGCCATACCGGTTATCACGTCGGCGGCTGGCAGGACGGCATTGACCAGGCCCGCGGCTTGACCGGCGGTGACATTGGCGATGCTGTAGTCACGCGCAGCAACGGCTCGCCAATATCTGGCCATGGCTTCTTCGCGATGGAGAATGTCGAGTTCGGTGTCCTCGAATTGGTCGGTGAGGGCGTTGCTTAGCACGCTCATCGTGTGTCCTTGCGGCCAGGGATAGCGCCGTAGCTGATCGTAGATAGTGGTGCGGCACATGTCGTCGCCAGTGGCCGCCAGCAGCGGGTCCCGCGCCTGCGGTGTGGATAACGCTTCGACCGTGGCGTAGAAGCGCGTACCGACCAATACCCCGGCGGCGCCCAACATCAACGCGGCGGCAAGGCCCCGGCCGTCGGCGATGCCCCCGGCGGCGATCACCGGGATATCAGTTCCCCGCGCGGTGACCAGGTCGACGATTTCGGGTACCAAGGTCAGGGTGGAACGTGGACCGTGGCCGTGCCCACCGGCCTCGGTGCCCTGAGCCACCAACACATCGGCGCCGACCTGCAGGGCTCGCTCGGCCTGGGTCCGGTTTTGGATCTGGCAGACCAACCGCGTTCCGGCGGACTTGATGGCGTCAGCGAAAACCGCGGGGTCCCCGAACGACAGCATCACCGCCACCGGCTCATACTGCAGCGCGAGGTCGAGCAGCTGCGGTTGGCGGGCCAAAGACCAGGTGATGAACCCGCAGCCCACCGGCGCTCCAGCGGCGAGATCGAACTGCCGGGCCAACCAATCCCGGTCCCCATAGCCGCCCCGATGAGGCCGAGTCCCCCTGCGCCACTTACCGCGGCAGCCAGCTCACCGCCGGCGATCAAGTCCATTGGCGCGGACACTATCGGATAGTCGATTCCGAACATCTGGCTAAAGGCCGTCGATAGCACCAC
>Rv0021c
TTACACGGCGCGC

### Creating a new MLST scheme with the novel alleles

To create a new MLST scheme with the novel alleles included, provide the original MLST scheme and the novel alleles FASTA file to the script 'create_new_scheme_with_novel.py' 

In [23]:
$PYTHON ../scripts/create_new_scheme_with_novel.py -h

usage: create_new_scheme_with_novel.py [-h] [-n NOVEL] [-o OUTPUT] [-i ID]
                                       [-ll {DEBUG,INFO,WARNING,ERROR,CRITICAL}]
                                       files [files ...]

Adds novel alleles to an existing MLST scheme.

positional arguments:
  files                 MLST Fasta files

optional arguments:
  -h, --help            show this help message and exit
  -n NOVEL, --novel NOVEL
                        FASTA with novel alleles.
  -o OUTPUT, --output OUTPUT
                        Output folder for new scheme.
  -i ID, --id ID        Start numbering new alleles on this value, later will
                        implement from last allele id +1.
  -ll {DEBUG,INFO,WARNING,ERROR,CRITICAL}, --loglevel {DEBUG,INFO,WARNING,ERROR,CRITICAL}
                        Set the logging level


So, to add the novel alleles from the previous step in the small MLST scheme from the initial example, we run:

In [24]:
$PYTHON ../scripts/create_new_scheme_with_novel.py MTB_scheme/*fa -o MTB_novel_scheme -n new_alleles.fa

06:12:31 PM (157 ms) -> INFO:Opening the novel alleles file ...
06:12:31 PM (183 ms) -> INFO:Opening the MLST schema and adding novel alleles ...
06:12:32 PM (1191 ms) -> INFO:Done.


We can see that the new scheme has more alleles on some loci:


In [25]:
grep -c ">" MTB_scheme/*fa | head -n5

MTB_scheme/Rv0014c.fa:396
MTB_scheme/Rv0015c.fa:331
MTB_scheme/Rv0016c.fa:289
MTB_scheme/Rv0017c.fa:298
MTB_scheme/Rv0018c.fa:353


In [26]:
grep -c ">" MTB_novel_scheme/*fa | head -n5

MTB_novel_scheme/Rv0014c.fa:398
MTB_novel_scheme/Rv0015c.fa:331
MTB_novel_scheme/Rv0016c.fa:290
MTB_novel_scheme/Rv0017c.fa:299
MTB_novel_scheme/Rv0018c.fa:354


### Run MentaLiST to create a new MLST database file
Similarly as before, but now with the new MLST scheme.

In [27]:
mentalist build_db --db mtb_novel_cgMLST.db -k 31 -f MTB_novel_scheme/*.fa

2017-11-23T18:12:41.855 - info: Opening FASTA files ... 
2017-11-23T18:12:55.432 - info: Combining results for each locus ...
2017-11-23T18:12:55.938 - info: Saving DB ...
2017-11-23T18:12:57.783 - info: Done!


Now, we can rerun the MLST calling phase with the new DB:


In [28]:
mentalist call -o sample_novel.call -s sample --db mtb_novel_cgMLST.db --kt 10 --output_votes --output_special sample.fastq.gz

2017-11-23T18:13:06.316 - info: Opening kmer database ... 
2017-11-23T18:13:09.648 - info: Opening fastq file(s) and counting kmers ... 
2017-11-23T18:14:09.145 - info: Voting for alleles ... 
2017-11-23T18:14:09.866 - info: Calling alleles and novel alleles ...
2017-11-23T18:14:14.2 - info: Writing output ...
2017-11-23T18:14:14.936 - info: Done.


Comparing this call with the previous, we can see that the novel alleles (marked as "N") have been called in the new output: 

In [29]:
# OLD:
cat sample.call

Sample	Rv0014c	Rv0015c	Rv0016c	Rv0017c	Rv0018c	Rv0019c	Rv0021c	Rv0022c	Rv0023	Rv0024	Rv0025	Rv0033	Rv0034	Rv0035	Rv0036c	Rv0037c	Rv0038	Rv0039c	Rv0040c	Rv0041	Rv0042c	Rv0043c	Rv0044c	Rv0045c	Rv0046c	Rv0047c	Rv0048c	Rv0049	Rv0050	Rv0051	Rv0052	Rv0053	Rv0054	Rv0055	Rv0056	Rv0058	Rv0059	Rv0060	Rv0061c	Rv0062	Rv0063a	Rv0063	Rv0065	Rv0066c	Rv0067c	Rv0068	Rv0069c	Rv0070c	Rv0075	Rv0076c	Rv0077c	Rv0078A	Rv0078B	Rv0078	Rv0079	Rv0080	Rv0081	Rv0082	Rv0084	Rv0085	Rv0086	Rv0087	Rv0089	Rv0090	Rv0091	Rv0092	Rv0097	Rv0098	Rv0099	Rv0767c	Rv0768	Rv1269c	ST	clonal_complex
sample	5	2	1	1	2	1	1	1	1	N	1	1	2	2	6	1	1	2	2	6	1	1	2	N	2	1	4	1	4	1	1	1	1	1	1	3	1	1+	2	1	1	N	1	1	1	2	1	1	1	1	1	3	1	1	1	1	1	1	1	1	3	1	1	3	1	4	1	1	1+	0	0	1/N?	0	0


In [30]:
# New:
cat sample_novel.call

Sample	Rv0014c	Rv0015c	Rv0016c	Rv0017c	Rv0018c	Rv0019c	Rv0021c	Rv0022c	Rv0023	Rv0024	Rv0025	Rv0033	Rv0034	Rv0035	Rv0036c	Rv0037c	Rv0038	Rv0039c	Rv0040c	Rv0041	Rv0042c	Rv0043c	Rv0044c	Rv0045c	Rv0046c	Rv0047c	Rv0048c	Rv0049	Rv0050	Rv0051	Rv0052	Rv0053	Rv0054	Rv0055	Rv0056	Rv0058	Rv0059	Rv0060	Rv0061c	Rv0062	Rv0063a	Rv0063	Rv0065	Rv0066c	Rv0067c	Rv0068	Rv0069c	Rv0070c	Rv0075	Rv0076c	Rv0077c	Rv0078A	Rv0078B	Rv0078	Rv0079	Rv0080	Rv0081	Rv0082	Rv0084	Rv0085	Rv0086	Rv0087	Rv0089	Rv0090	Rv0091	Rv0092	Rv0097	Rv0098	Rv0099	Rv0767c	Rv0768	Rv1269c	ST	clonal_complex
sample	5	2	1	1	2	1	1	1	1	234	1	1	2	2	6	1	1	2	2	6	1	1	2	192	2	1	4	1	4	1	1	1	1	1	1	3	1	1+	2	1	1	331	1	1	1	2	1	1	1	1	1	3	1	1	1	1	1	1	1	1	3	1	1	3	1	4	1	1	1+	0	0	1/N?	0	0
